In [ ]:
%pip install openai qdrant_client langchain tiktoken fastembed -q

### Импорты библиотек

In [311]:
import json
import openai
import concurrent.futures
from typing import List, Dict
from langchain.text_splitter import TokenTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
from concurrent.futures import ThreadPoolExecutor

### Заданные константы

In [312]:
# Ваш хост
HOST = '127.0.0.1'

# Эмбеддинг
EMBEDDING_BASE_URL = f"http://{HOST}:9001/v1"
EMBEDDING_MODEL_NAME = "bge-m3"

# Векторная база данных
QDRANT_URL = f"http://{HOST}:6333"
COLLECTION_NAME = "TEST"

# LLM
BASE_URL = "http://{HOST}:9000/v1"
MODEL_NAME = "Qwen2.5-14B-Instruct-GPTQ-Int4"

# Количество потоков для параллельности обработки
EMBEDDING_THREADS = 8
CREATE_SLIDES_THREADS = 6

### Получение эмбеддингов

In [313]:
embedding_client = openai.OpenAI(api_key="EMPTY", base_url=EMBEDDING_BASE_URL)

In [314]:
def get_text_embedding(text: str) -> List:
    embedding_response = embedding_client.embeddings.create(
        model=EMBEDDING_MODEL_NAME,
        input=text
    )
    embedding = embedding_response.data[0].embedding
    return embedding

In [316]:
len(get_text_embedding("Тест эмбеддинг доступности эмбеддинг модели"))

1024

### Пример запроса пользователя и содержания файла

In [273]:
user_prompt = 'Сделай презентацию про функционал и производительность Атом.Око'

In [317]:
def read_text_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

file_content_1 = read_text_file('test_data/atom_search_1.txt')
file_content_2 = read_text_file('test_data/atom_search_2.txt')
file_content_3 = read_text_file('test_data/atom_search_3.txt')

content_text = file_content_3
print("Количество символов в текстовом документе:", len(content_text))

Количество символов в текстовом документе: 12959


### Загрузка в векторную базу данных [Qdrant](https://qdrant.tech/)

In [318]:
qdrant_client = QdrantClient(url=QDRANT_URL)

In [319]:
def split_text_into_chunks(text: str, chunk_size: int, chunk_overlap=50) -> List[str]:
    text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_text(text)
    return chunks


chunk_size = 200
chunks = split_text_into_chunks(content_text, chunk_size)
print(len(chunks))

92


In [320]:
if not qdrant_client.collection_exists(COLLECTION_NAME):
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=1024, distance=Distance.DOT),
    )

In [321]:
def embed_chunk(chunk: str) -> List:
    return get_text_embedding(chunk)

vectors = []
with ThreadPoolExecutor(max_workers=EMBEDDING_THREADS) as executor:  # Замените N на желаемое количество потоков
    embeddings = list(executor.map(embed_chunk, chunks))

for i, vector in enumerate(embeddings):
    vectors.append(PointStruct(id=i + 1, vector=vector, payload={"chunk": chunks[i]}))

operation_info = qdrant_client.upsert(
    collection_name=COLLECTION_NAME,
    wait=True,
    points=vectors,
)
print(operation_info)

operation_id=1 status=<UpdateStatus.COMPLETED: 'completed'>


In [324]:
def get_relevant_docs(user_query: str) -> List:
    search_result = qdrant_client.query_points(
        collection_name=COLLECTION_NAME,
        query=get_text_embedding(user_query),
        with_payload=True,
        limit=5
    ).points

    found_texts = []
    for result in search_result:
        found_texts.append(
            result.payload.get("chunk")
        )

    return found_texts

In [325]:
get_relevant_docs('Функционал')

['ения информации из документов, проверки данных и автоматизации документооборота в различных сферах бизнеса 【10】.\n\n2. Функциональные возможности\nРаспознавание документов: Обрабатывает с',
 'остоятельно добавлять новые типы документов и настраивать логику обработки 【10】.\nВопросно-ответная система: Функция «Поговори с документом» позволяет пользователю загружать файл и запрашив',
 'ефон. Проблема решается быстро и эффективно.\nЛояльность:\nПользователь становится постоянным клиентом, продлевая подписку на премиум-функции.\nОн рекомендует "Атом.Око" друзьям и коллегам',
 'во: распознавание заявлений и уведомлений сотрудников4.\n\nАрхивная деятельность: оцифровка и систематизация больших объемов документов45.\n\nЛогистика и закупки: проверка комплектности и к',
 '.\nОн рекомендует "Атом.Око" друзьям и коллегам, возможно, даже участвует в партнерской программе для получения скидок.\n7. Стадия: Расширение использования\nНовые функции: Компания регуля']

### Генерация с помощью LLM

In [326]:
def extract_llm_answer(llm_answer: str) -> List | Dict:
    # Извлекаем текст между ```json и ```
    json_text = llm_answer.split('```json')[1].split('```')[0].strip()
    return json.loads(json_text)

In [327]:
llm_client = openai.OpenAI(api_key="EMPTY", base_url=BASE_URL)

#### Суммаризация документа

In [328]:
SUMMARY_TEXT_CONTENT_SYSTEM_PROMPT = "Ты эксперт в создании кратких и точных резюме документов. Содержание должно отражать суть документа и состоять не больше, чем из 3 предложений."
SUMMARY_TEXT_CONTENT_USER_PROMPT_TEMPLATE = "Пожалуйста, составь краткое содержание для следующего документа: {text_content}\n\nПри составлении учитывай, что хочет пользователь. Запрос пользователя: {user_query}\nКраткое содержание:"

In [329]:
summary_threashold = 5000  # Максимальная длина текста для суммаризации

# Суммаризируем в несколько потоков
if len(content_text) > summary_threashold:
    chunks = split_text_into_chunks(content_text, summary_threashold, chunk_overlap=100)
    with ThreadPoolExecutor() as executor:
        chat_responses = list(executor.map(lambda chunk: llm_client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": SUMMARY_TEXT_CONTENT_SYSTEM_PROMPT},
                {"role": "user", "content": SUMMARY_TEXT_CONTENT_USER_PROMPT_TEMPLATE.format(text_content=chunk, user_query=user_prompt)},
            ],
            temperature=0
        ), chunks))
    
    summary_content = " ".join(response.choices[0].message.content for response in chat_responses)

# Суммаризируем в один поток
else:
    chat_response = llm_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SUMMARY_TEXT_CONTENT_SYSTEM_PROMPT},
            {"role": "user", "content": SUMMARY_TEXT_CONTENT_USER_PROMPT_TEMPLATE.format(text_content=content_text, user_query=user_prompt)},
        ],
        temperature=0
    )
    summary_content = chat_response.choices[0].message.content

In [330]:
print(summary_content)

Атом.Око — это программное обеспечение для автоматизации работы с документами, разработанное компанией «Гринатом». Оно использует технологии машинного обучения для точного распознавания и анализа текста, изображений и таблиц, что позволяет значительно ускорить обработку документов и снизить временные затраты. Продукт поддерживает интеграцию с различными корпоративными системами и обеспечивает высокую точность и безопасность при работе с конфиденциальными данными. Атом.Око представляет собой систему для автоматизации документооборота с возможностью гибкой настройки и интеграции с другими ИТ-системами. Она обеспечивает высокую точность распознавания и производительность, адаптируясь к различным типам документов и нагрузкам. Система уже успешно используется крупными компаниями, автоматизируя обработку документов и снижая временные и финансовые затраты. "Атом.Око" - это устройство, обеспечивающее мониторинг состояния глаз и осанки пользователя. Оно совместимо с различными платформами и пре

In [331]:
len(summary_content)

1253

#### Генерация заголовков слайдов

In [332]:
CREATE_SLIDES_PLAN_ANSWER_EXAMPLE = '''```json\n[{"title": "...", "info": "...", "use_list": true}, {"title": "...", "info": "...", "use_list": false}]\n```
Для каждого слайда используй следующие поля:
- "title": Название слайда.
- "info": Краткая информация из контекста в 1 предложении, которая будет расположена на слайде.
- "use_list": Флаг использования списка, true - использовать список на слайде, false - не использовать список.
'''
CREATE_SLIDES_PLAN_SYSTEM_PROMPT = f"Ты креативный создатель презентаций. Твоя задача придумывать заголовки к слайдам презентации. Ответ дай в формате JSON:{CREATE_SLIDES_PLAN_ANSWER_EXAMPLE}"
CREATE_SLIDES_PLAN_USER_PROMPT_TEMPLATE = "Тебе дан следующий контекст: {summary_content}\nСделай презентацию по запросу пользователя: {user_query}\nНазвание слайдов в виде JSON:"

In [335]:
chat_response = llm_client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": CREATE_SLIDES_PLAN_SYSTEM_PROMPT},
        {"role": "user", "content": CREATE_SLIDES_PLAN_USER_PROMPT_TEMPLATE.format(summary_content=summary_content, user_query=user_prompt)},
    ],
    temperature=0
)
slides_titles = chat_response.choices[0].message.content
slides_titles_array = extract_llm_answer(slides_titles)
slides_titles_array

[{'title': 'Введение в Атом.Око',
  'info': 'Атом.Око — это программное обеспечение для автоматизации работы с документами, разработанное компанией «Гринатом».',
  'use_list': False},
 {'title': 'Технологии и возможности',
  'info': 'Программное обеспечение использует технологии машинного обучения для точного распознавания и анализа текста, изображений и таблиц.',
  'use_list': True},
 {'title': 'Преимущества использования',
  'info': 'Атом.Око позволяет значительно ускорить обработку документов и снизить временные затраты.',
  'use_list': True},
 {'title': 'Интеграция и безопасность',
  'info': 'Продукт поддерживает интеграцию с различными корпоративными системами и обеспечивает высокую точность и безопасность при работе с конфиденциальными данными.',
  'use_list': False},
 {'title': 'Гибкость и настройка',
  'info': 'Атом.Око представляет собой систему для автоматизации документооборота с возможностью гибкой настройки и интеграции с другими ИТ-системами.',
  'use_list': False},
 {'ti

#### Генерация контента для каждого слайда на основе релевантных документов из векторной базы данных

In [336]:
CREATE_SLIDES_CONTENT_ANSWER_EXAMPLE = '''```json\n{"title": "...", "text": "..."}\n```'''
CREATE_SLIDES_CONTENT_INSTRUCTIONS = '''При написании контента к слайду, ты можешь использовать следующие поля:
- "title": название слайда.
- "text": основной текст слайда - указывай обязательно для каждого слайда.'''
CREATE_SLIDES_CONTENT_LIST = '''- "list": ["...", "..."] - перечисление пунктов или создание списков для презентации (bullet points).'''
CREATE_SLIDES_CONTENT_SYSTEM_PROMPT = f"Ты креативный создатель презентаций. Твоя задача - создавать содержимое для слайдов презентации. Ответ должен быть в формате JSON:{CREATE_SLIDES_CONTENT_ANSWER_EXAMPLE}\n{CREATE_SLIDES_CONTENT_INSTRUCTIONS}"
CREATE_SLIDES_CONTENT_USER_PROMPT_TEMPLATE = "Тебе дана следующая информация: {relevant_content}\nТебе нужно составить содержание для слайда с названием: {slide_topic}. Содержание слайда в виде JSON:"

In [338]:
slides_content = []

def process_slide(slide_topic: Dict) -> Dict:
    topic = slide_topic['title']
    slide_info = slide_topic['info']
    use_list_on_slide = slide_topic['use_list']

    system_prompt = CREATE_SLIDES_CONTENT_SYSTEM_PROMPT + CREATE_SLIDES_CONTENT_LIST if use_list_on_slide else CREATE_SLIDES_CONTENT_SYSTEM_PROMPT

    docs = get_relevant_docs(slide_info)
    relevant_content = '\n'.join(docs)
    chat_response = llm_client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt },
            {"role": "user", "content": CREATE_SLIDES_CONTENT_USER_PROMPT_TEMPLATE.format(relevant_content=relevant_content, slide_topic=topic)},
        ]
    )
    slide_content = chat_response.choices[0].message.content
    return extract_llm_answer(slide_content)

with concurrent.futures.ThreadPoolExecutor(max_workers=CREATE_SLIDES_THREADS) as executor:
    slides_content = list(executor.map(process_slide, slides_titles_array))

In [339]:
print("Создалась презентация по запросу пользователя:", user_prompt)

for i, slide_content in enumerate(slides_content):
    print(f"\nSlide {i+1}:")
    print(slide_content.get('title'))
    print(slide_content.get('text'))
    if slide_content.get('list'):
        for element in slide_content.get('list'):
            print(" - ", element)

Создалась презентация по запросу пользователя: Сделай презентацию про функционал и производительность Атом.Око

Slide 1:
Введение в Атом.Око
Атом.Око — это инновационное программное обеспечение для распознавания, анализа и обработки документов, разработанное компанией «Гринатом». Оно позволяет автоматизировать документооборот с высокой точностью, гибкими настройками и возможностью интеграции с другими информационными системами.

Slide 2:
Технологии и возможности
Атом.Око использует передовые технологии машинного обучения и искусственного интеллекта для обеспечения высокой точности и скорости обработки данных.
 -  Обрабатывает сканированные документы и фотографии, извлекая текст даже при плохом освещении или сложных углах съемки.
 -  Проверяет данные и точно извлекает информацию из документов.
 -  Внедряется в различные отрасли, такие как бухгалтерия, кадровое делопроизводство, закупки и архивная деятельность.

Slide 3:
Преимущества использования Атом.Око
Атом.Око предлагает ряд значимы

#### Чистим векторную бд

In [340]:
qdrant_client.delete_collection(collection_name=COLLECTION_NAME)

True